In [ ]:
import numpy as np
import itertools
import cv2
from math import exp
from random import seed
from random import random
import pickle

## Type of Window

In [ ]:
W = np.array([[0,1,0],[1,1,1],[0,1,0]]) # Janela EM cruz
W_size = W.shape[0]
inc = int(round(W_size/2-0.1,0))

## Prepare Training Data

In [ ]:
X = []
y = []
for img in range(1,15):
    s = str(img)
    s = s.zfill(2)
    img_in = cv2.imread('./X/img'+s+'.jpg', cv2.IMREAD_GRAYSCALE)
    img_out = cv2.imread("./y/img"+s+".jpg", cv2.IMREAD_GRAYSCALE) # bordas
    #y = cv2.imread("./data_out_sp/img"+s+".jpg", cv2.IMREAD_GRAYSCALE) # sal-pimenta
    (T, img_in) = cv2.threshold(img_in, 100, 255, cv2.THRESH_BINARY)
    (T, img_out) = cv2.threshold(img_out, 100, 255, cv2.THRESH_BINARY)
    if img > 6:
        img_in[(img_in==0)]=1
        img_out[(img_out==0)]=1
        
        img_in[(img_in==255)]=0
        img_out[(img_out==255)]=0
        
    else:
        img_in[(img_in==255)]=1
        img_out[(img_out==255)]=1
    
    
    img_in = img_in.astype(int)
    img_out = img_out.astype(int)
    
    img_inl = np.c_[np.zeros([img_in.shape[0],inc], dtype=int),img_in, np.zeros([img_in.shape[0],inc], dtype=int)]
    img_inl = np.r_[np.zeros([inc,img_inl.shape[1]], dtype=int),img_inl, np.zeros([inc,img_inl.shape[1]], dtype=int)]
    
    for i in range(img_in.shape[0]):
        for j in range(img_in.shape[1]):

            px = []
            for k in range(W.shape[0]):
                for r in range(W.shape[1]):
                    if W[k,r]==1:
                        px.append(img_inl[i-inc+1+k,j-inc+1+r])
        
            X.append(px)
            y.append(img_out[i,j])
            

In [ ]:
y = np.array(y)
X = np.array(X)
X.shape, y.shape

In [ ]:
# Change 0 to -1 to the neural network
X[X==0]=-1

In [ ]:
y = y.reshape(1,-1)

X = np.transpose(X)
print(f"X: {X.shape}, y: {y.shape}")

In [ ]:
print(f"X: {X.shape}, y: {y.shape}")

In [ ]:
X

In [ ]:
y

## Neural Network functions

In [ ]:
layer_dims = [5,32,1]

In [ ]:
def initialize_parameters(layer_dims, fixed_first_layer = True):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        if (l==1)and(fixed_first_layer):
            parameters['W' + str(l)] = np.array([list(i) 
                                                 for i in itertools.product([-1, 1]
                                                 , repeat=layer_dims[l - 1])])
            parameters['b' + str(l)] = np.array([-4]*layer_dims[l]).reshape(layer_dims[l],1)
        else:
            parameters['W' + str(l)] = np.random.randn(layer_dims[l],
                                                   layer_dims[l - 1]) * 0.1
            parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

        assert(parameters['W' + str(l)].shape ==
               (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

In [ ]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, cache

In [ ]:
def L_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)

    return Z, cache


def L_activation_forward(A_prev, W, b):
    Z, linear_cache = L_forward(A_prev, W, b)
    A, activation_cache = sigmoid(Z)
    cache = (linear_cache, activation_cache)

    return A, cache


def L_model_forward(X, parameters):
    A = X
    caches = []
    L = len(parameters) // 2

    for l in range(1, L+1):
        A_prev = A

        A, cache = L_activation_forward(
            A_prev, parameters["W" + str(l)], parameters["b" + str(l)])
        caches.append(cache)

    return A, caches


In [ ]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -(np.sum(Y * np.log(AL) + (1.0 - Y) * np.log(1.0 - AL))) / m # Logloss error
    cost = np.squeeze(cost)

    return cost


In [ ]:
def L_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = np.dot(dZ, A_prev.T) / m # delta W = gradient * neurons_inputs
    db = np.sum(dZ, axis=1, keepdims=True) / m # delta bias = sigmoid derivative median
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db
    
def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s) # gradient = sigmoid derivative * logloss derivative

    return dZ


In [ ]:
def L_activation_backward(dA, cache):
    linear_cache, activation_cache = cache

    dZ = sigmoid_backward(dA, activation_cache)
    dA_prev, dW, db = L_backward(dZ, linear_cache)

    return dA_prev, dW, db


In [ ]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) 
    Y = Y.reshape(AL.shape)

    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # logloss derivative

    current_cache = caches[L - 1]
    dA_prev_temp, dW_temp, db_temp = L_activation_backward(dAL, current_cache)
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp


    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = L_activation_backward(
            grads["dA" + str(l + 1)], current_cache)
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads


In [ ]:
def update_parameters(params, grads, learning_rate, fixed_first_layer):
    parameters = params.copy()
    L = len(parameters) // 2 
    for l in range(L):
        # W_new = W_old - learning_rate * gradient_W
        # b_new = b_old - learning_rate * gradient_b
        
        if fixed_first_layer and l==0:
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] 
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] 
        else:
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - \
                learning_rate * grads["dW" + str(l + 1)]
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - \
                learning_rate * grads["db" + str(l + 1)]

    return parameters


In [ ]:
def nn_model_train(X, Y, layers_dims, learning_rate=0.1, num_iterations = 20000
                   , print_cost = True, fixed_first_layer = True, ini_param = False):
    np.random.seed(1)
    costs = [] # keep track of cost

    # Parameters initialization.
    if ini_param:
        with open("parameters_v9.pkl","rb") as r:
            parameters = pickle.load(r)
    else:
        parameters = initialize_parameters(layers_dims, fixed_first_layer)


    for i in range(0, num_iterations):
        AL, caches = L_model_forward(X, parameters) # Calculate the output of the network -- forward propagation
        cost = compute_cost(AL, Y) # Calculate the Logloss error
        grads = L_model_backward(AL, Y, caches) # Calculate the Gradient
        parameters = update_parameters(parameters, grads, learning_rate, fixed_first_layer)

        if i % 1000 == 0:
            learning_rate = learning_rate / 2 # After 2.000 iterarions, divide learning_rate by 2
        if i % 2000 == 0:
            learning_rate = learning_rate / 2 # After 2.000 iterarions, divide learning_rate by 2

        if print_cost and i % 10 == 0 or i == num_iterations - 1:
            print(f"Cost after iteration {i}: {np.squeeze(cost)}")
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)

    return parameters, costs


## Train

In [ ]:
layers_dims = [5, 32, 1] # 5 inputs / 1 hidden layer with 2^5 = 32 neurons / 1 output layer

In [ ]:
parameters, costs = nn_model_train(
    X, y, layers_dims, learning_rate = 0.2, num_iterations=500, print_cost=True, fixed_first_layer = True, ini_param =True)

In [ ]:
import pickle
f = open("parameters_v10.pkl","wb")
pickle.dump(parameters,f)
f.close()

## Apply Network

In [ ]:
def predict(X, y, parameters):
    m = X.shape[1]
    n = len(parameters) // 2 
    p = np.zeros((1, m))

    probas, caches = L_model_forward(X, parameters)

    for i in range(0, probas.shape[1]):
        if probas[0, i] > 0.5:
            p[0, i] = 1
        else:
            p[0, i] = 0
    print("Accuracy: " + str(np.sum((p == y)/m)))

    return p

def apply(X, parameters):
    m = X.shape[1]
    n = len(parameters) // 2 
    p = np.zeros((1, m))

    probas, caches = L_model_forward(X, parameters)

    for i in range(0, probas.shape[1]):
        if probas[0, i] > 0.57:
            p[0, i] = 255
        else:
            p[0, i] = 0

    return p

In [ ]:
predict(X, y, parameters)

In [ ]:
for img in range(1,17):
    s = str(img)
    s = s.zfill(2)
    x = cv2.imread('./X/img'+s+'.jpg', cv2.IMREAD_GRAYSCALE)
    #cv2.imshow('image',x)
    #cv2.waitKey(0)
    
    (T, x) = cv2.threshold(x, 100, 255, cv2.THRESH_BINARY)
    x[(x==255)]=1
    
    x = x.astype(int)
    
    Xl = np.c_[np.zeros([x.shape[0],inc], dtype=int),x, np.zeros([x.shape[0],inc], dtype=int)]
    Xl = np.r_[np.zeros([inc,Xl.shape[1]], dtype=int),Xl, np.zeros([inc,Xl.shape[1]], dtype=int)]
    
    z = np.zeros(x.shape, dtype=int)
    
    for i in range(x.shape[0]):
        row = []
        for j in range(x.shape[1]):

            px = []
            for k in range(W.shape[0]):
                for r in range(W.shape[1]):
                    if W[k,r]==1:
                        px.append(Xl[i-inc+1+k,j-inc+1+r])
            row.append(px)
            
        row = np.transpose(np.array(row))
        result = apply(row,parameters)
        
                        
        z[i,:] = result
    
    cv2.imwrite('./data_out/img'+s+'.jpg', z) 